In [40]:
import pandas as pd
import numpy as np

df = pd.read_csv("10_05_2021.csv", dtype={'work_postal':'str'})
df['end_date'] = df['end_date'].fillna(pd.to_datetime('2021-11-01'))

#===================== floats to ints ===============================

#round the floats, then convert to integer
df["comprate"] = df["comprate"].round(0)
df["comprate"] = df["comprate"].astype(np.int64)
df["duration"] = df["duration"].round(0)
df["duration"] = df["duration"].astype(np.int64)

#======================= city entries ==============================

checkUniques = {} #dictionary that will contain unique city entries
cityCols = ['work_city','home_city']

for col in cityCols:
    df[col] = df[col].str.upper() # make all city entries uppercase

#replace errors that I found using the testing code below (cities.csv)
for col in cityCols:
    df[col] = df[col].replace(to_replace ="CHINO HILLLS", value = "CHINO HILLS", regex = True)
    df[col] = df[col].replace(to_replace ="CORONA HILLS", value = "CORONA", regex = True)
    df[col] = df[col].replace(to_replace ="DESERT HOT SPRUNGS", value = "DESERT HOT SPRINGS", regex = True)
    df[col] = df[col].replace(to_replace ="HEMET CA", value = "HEMET", regex = True)
    df[col] = df[col].replace(to_replace ="EASTEVALE", value = "EASTVALE", regex = True)
    df[col] = df[col].replace(to_replace ="JURUAPA VALLEY", value = "JURUPA VALLEY", regex = True)
    df[col] = df[col].replace(to_replace ="^JURUPA$", value = "JURUPA VALLEY", regex = True)
    df[col] = df[col].replace(to_replace ="MORENO VALLY", value = "MORENO VALLEY", regex = True)
    df[col] = df[col].replace(to_replace ="MORNEO VALLEY", value = "MORENO VALLEY", regex = True)
    df[col] = df[col].replace(to_replace ="N PALM SPRINGS", value = "PALM SPRINGS", regex = True)
    df[col] = df[col].replace(to_replace ="NUEVEO", value = "NUEVO", regex = True)
    df[col] = df[col].replace(to_replace ="RANCH CUCAMONGA", value = "RANCHO CUCAMONGA", regex = True)
    df[col] = df[col].replace(to_replace ="RANCHO CUCAMONGA", value = "RANCHO CUCAMONGA", regex = True)
    df[col] = df[col].replace(to_replace ="RANCHO CUCAMONGA,", value = "RANCHO CUCAMONGA", regex = True)
    df[col] = df[col].replace(to_replace ="RIVESIDE", value = "RIVERSIDE", regex = True)
    df[col] = df[col].replace(to_replace ="SAN BERNADINO", value = "SAN BERNARDINO", regex = True)
    df[col] = df[col].replace(to_replace ="TWENTY-NINE PALMS", value = "TWENTYNINE PALMS", regex = True)
    df[col] = df[col].replace(to_replace ="WIDOMAR", value = "WILDOMAR", regex = True)
    df[col] = df[col].replace(to_replace ="nan", value = "", regex = True)

#=================================== zip code formats =====================================

postalCols = ['home_postal','work_postal']

for key in postalCols:
    df[key] = df[key].replace(to_replace = '-[0-9]+', value = "", regex = True) #delete the extra digits
    df[key] = df[key].replace(to_replace = '[^0-9]', value = "", regex = True) #delete entries that are not digits only
    df[key] = df[key].replace(to_replace = '^[0-9]{1,4}$', value = "", regex = True) #delete entries that are less than 5 digits
    df[key] = df[key].replace(to_replace = '^[0-9]{6,}$', value = "", regex = True) #more than five digits

#==================================== fill in event column to make machine learning model easier to work with=====================



#==================================== tests and checking =================================

#--------- write unique city names to file to look for mistakes/duplicates-------------
for i in range(len(df["work_city"])):
    checkUniques[df["work_city"][i]] = "temp"

for i in range(len(df["home_city"])):
    checkUniques[df["home_city"][i]] = "temp"

#write the namse into a csv just so I can open it in excel, sort alphabetically, and check for duplicates
with open('cities.csv', 'w') as f:
    for key in checkUniques.keys():
        f.write("%s\n"%(key))

#------------------------------- check zip codes for right format------------------------

for key in postalCols:
    for i in range(len(df)):
        if pd.isna(df[key][i]):
            continue
        if df[key][i] == "":
            continue
        if len(df[key][i]) != 5:
            print(key, i, df[key][i])
            print([df[key][i]])
            print(len(df[key][i]))
            break
  
df.to_csv("cleaned.csv")


##to do. make new column called status. will be filled with 4 vars: working, terminated, retired, and unknown. 
#find a way to integrate 